In [13]:
import pandas as pd              # For working with data
import numpy as np               # For numerical operations
import matplotlib.pyplot as plt  # For visualizations
import seaborn as sns            # For prettier visualizations

from sklearn.model_selection import train_test_split  # To split the data
from sklearn.tree import DecisionTreeClassifier        # Our ML model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # To evaluate our model


In [17]:
honeypot = pd.read_csv('./data/honeypot.csv', on_bad_lines='skip')


In [19]:
honeypot.head()

,SrcId,Seq,StartTime,LastTime,Dur,SrcStartTime,SrcLastTime,SrcDur,DstStartTime,DstLastTime,...,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat,Inode,sMaxPktSz,sMinPktSz,dMaxPktSz,dMinPktSz
0,0.0.0.0,1,2021/04/23 02:00:02.302762,2021/04/23 02:00:41.489839,39.187077,2021/04/23 02:00:41.489839,2021/04/23 02:00:41.489839,0.000000,2021/04/23 02:00:02.302762,2021/04/23 02:00:02.302762,...,3.591833e+09,NaN,0.0,0.0,0.0,NaN,74.0,74.0,54.0,54.0
1,0.0.0.0,1,2021/04/23 02:00:41.489839,2021/04/23 02:01:58.865101,77.375259,2021/04/23 02:00:41.489839,2021/04/23 02:01:58.865101,77.375259,2021/04/23 02:00:41.489866,2021/04/23 02:01:20.257937,...,2.554704e+09,NaN,0.0,0.0,0.0,NaN,74.0,74.0,54.0,54.0
2,0.0.0.0,1,2021/04/23 02:01:58.865101,2021/04/23 02:03:15.553545,76.688446,2021/04/23 02:01:58.865101,2021/04/23 02:03:15.553545,76.688446,2021/04/23 02:01:58.865128,2021/04/23 02:02:37.521042,...,9.464667e+08,NaN,0.0,0.0,0.0,NaN,74.0,74.0,54.0,54.0
3,0.0.0.0,1,2021/04/23 02:03:15.553545,2021/04/23 02:04:31.585332,76.031784,2021/04/23 02:03:15.553545,2021/04/23 02:04:31.585332,76.031784,2021/04/23 02:03:15.553575,2021/04/23 02:03:53.790391,...,4.905424e+08,NaN,0.0,0.0,0.0,NaN,74.0,74.0,54.0,54.0
4,0.0.0.0,1,2021/04/23 02:04:31.585332,2021/04/23 02:05:47.200660,75.615326,2021/04/23 02:04:31.585332,2021/04/23 02:05:47.200660,75.615326,2021/04/23 02:04:31.585360,2021/04/23 02:05:09.553905,...,4.082257e+09,NaN,0.0,0.0,0.0,NaN,74.0,74.0,54.0,54.0


In [18]:
print(honeypot.shape)
honeypot.head()
honeypot.columns.tolist()


(8593, 118)


['SrcId',
 'Seq',
 'StartTime',
 'LastTime',
 'Dur',
 'SrcStartTime',
 'SrcLastTime',
 'SrcDur',
 'DstStartTime',
 'DstLastTime',
 'DstDur',
 'SRange',
 'Trans',
 'Flgs',
 'StdDev',
 'SrcAddr',
 'Dir',
 'DstAddr',
 'Proto',
 'Sport',
 'Dport',
 'sCo',
 'dCo',
 'sTos',
 'dTos',
 'sDSb',
 'dDSb',
 'sTtl',
 'dTtl',
 'sHops',
 'dHops',
 'sIpId',
 'dIpId',
 'TotPkts',
 'SrcPkts',
 'DstPkts',
 'TotBytes',
 'SrcBytes',
 'DstBytes',
 'TotAppByte',
 'SAppBytes',
 'DAppBytes',
 'Load',
 'SrcLoad',
 'DstLoad',
 'Rate',
 'SrcRate',
 'DstRate',
 'Loss',
 'SrcLoss',
 'DstLoss',
 'pLoss',
 'pSrcLoss',
 'pDstLoss',
 'sEnc',
 'dEnc',
 'SrcMac',
 'DstMac',
 'sMpls',
 'dMpls',
 'sVlan',
 'dVlan',
 'sVid',
 'dVid',
 'sVpri',
 'dVpri',
 'SIntPkt',
 'DIntPkt',
 'SIntPktAct',
 'DIntPktAct',
 'SIntPktIdl',
 'DIntPktIdl',
 'SIntPktMax',
 'SIntPktMin',
 'DIntPktMax',
 'DIntPktMin',
 'SIPActMax',
 'SIPActMin',
 'DIPActMax',
 'DIPActMin',
 'SIPIdlMax',
 'SIPIdlMin',
 'DIPIdlMax',
 'DIPIdlMin',
 'SrcJitter',
 'Dst

In [20]:
print(honeypot.head(3))

     SrcId  Seq                   StartTime                    LastTime  \
0  0.0.0.0    1  2021/04/23 02:00:02.302762  2021/04/23 02:00:41.489839   
1  0.0.0.0    1  2021/04/23 02:00:41.489839  2021/04/23 02:01:58.865101   
2  0.0.0.0    1  2021/04/23 02:01:58.865101  2021/04/23 02:03:15.553545   

         Dur                SrcStartTime                 SrcLastTime  \
0  39.187077  2021/04/23 02:00:41.489839  2021/04/23 02:00:41.489839   
1  77.375259  2021/04/23 02:00:41.489839  2021/04/23 02:01:58.865101   
2  76.688446  2021/04/23 02:01:58.865101  2021/04/23 02:03:15.553545   

      SrcDur                DstStartTime                 DstLastTime  ...  \
0   0.000000  2021/04/23 02:00:02.302762  2021/04/23 02:00:02.302762  ...   
1  77.375259  2021/04/23 02:00:41.489866  2021/04/23 02:01:20.257937  ...   
2  76.688446  2021/04/23 02:01:58.865128  2021/04/23 02:02:37.521042  ...   

     SrcTCPBase  DstTCPBase  TcpRtt SynAck  AckDat Inode sMaxPktSz sMinPktSz  \
0  3.591833e+09      

In [21]:
# Drop columns with mostly NaNs or not useful for our analysis
columns_to_drop = [
    'Inode', 'DstTCPBase', 'Flgs', 'SRange', 'Trans', 
    'StdDev', 'SrcId', 'Seq', 'SrcTCPBase', 'sTos', 'dTos'
]

honeypot_cleaned = honeypot.drop(columns=columns_to_drop, errors='ignore')


In [22]:
#  Convert timestamps to datetime
honeypot_cleaned['StartTime'] = pd.to_datetime(honeypot_cleaned['StartTime'], errors='coerce')
honeypot_cleaned['LastTime'] = pd.to_datetime(honeypot_cleaned['LastTime'], errors='coerce')


In [23]:
honeypot_cleaned = honeypot_cleaned.fillna(0)


In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
honeypot_cleaned['SrcAddr'] = le.fit_transform(honeypot_cleaned['SrcAddr'])
honeypot_cleaned['DstAddr'] = le.fit_transform(honeypot_cleaned['DstAddr'])
honeypot_cleaned['Proto'] = le.fit_transform(honeypot_cleaned['Proto'])
